In [2]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 774.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 2.8 MB/s eta 0:00:00
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Using cached frozendict-2.3.8-cp310-cp310-macosx_11_0_arm64.whl (35 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    Uninstalling requests-2.28.1:
      Successfully uninstalled requests-2.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
spyder 5.4.1 requires pyqt5<5.16, which is not installed.
spyder 5.4.1 requires pyqtwebengine<5.16, which is not installed.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo

In [4]:
!pip install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 2.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.2 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [48]:
import yfinance as yf
import talib
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from textblob import TextBlob
from sklearn.ensemble import RandomForestRegressor

symbol = "AAPL"  # Stock symbol

# Get data
stock = yf.Ticker(symbol)
hist = stock.history(period="max")

# Technical indicators
hist['rsi'] = talib.RSI(hist['Close'])

macd, macdsignal, macdhist = talib.MACD(hist['Close'])
hist['macd'] = macd

hist['sma50'] = talib.SMA(hist['Close'], timeperiod=50)
hist['sma200'] = talib.SMA(hist['Close'], timeperiod=200)

bbands = talib.BBANDS(hist['Close'])
upper, middle, lower = bbands
hist['bb_upper'] = upper

# Fundamentals
pe_ratio = stock.info['trailingPE']
pb_ratio = stock.info['priceToBook']

# News sentiment
news_title = stock.news[0]['title']
news = TextBlob(news_title)
sentiment = news.sentiment.polarity

# Merge fundamentals and sentiment
sentiment_series = pd.Series(sentiment, index=hist.index, name='sentiment')
pb_series = pd.Series(pb_ratio, index=hist.index, name='pb_ratio')
pe_series = pd.Series(pe_ratio, index=hist.index, name='pe_ratio')

hist = hist.join([sentiment_series, pb_series, pe_series])

# Remove rows with missing values
hist.dropna(inplace=True)

# Separate features and target
X = hist[['rsi', 'macd', 'sma50', 'sma200', 'bb_upper', 'pe_ratio', 'pb_ratio', 'sentiment']]
y = hist['Close']

# Ensemble model
ensemble = RandomForestRegressor()

# Fit the ensemble model
ensemble.fit(X, y)

# Print recommendation
current_features = X.iloc[-1].values.reshape(1, -1)
predicted_price = ensemble.predict(current_features)[0]
current_price = y.iloc[-1]
print("Predicted price:", predicted_price)
print("Current price: {:.2f}".format(current_price))

# Forecast returns for different horizons
returns = {}

for horizon in [1, 2, 5, 7]:
    feature_horizon = X.iloc[-horizon].values.reshape(1, -1)

    # Make prediction
    pred = ensemble.predict(feature_horizon)[0]

    # Calculate return
    ret = (pred / current_price - 1) * 100

    returns[horizon] = ret

# Print recommendation
if predicted_price < current_price:
    print("SELL - Expected Return: {:.2f}% 1 week later".format(returns[7]))
else:
    print("BUY - Expected Return: +{:.2f}% 1 week later".format(returns[7]))

# Hold recommendation
already_bought = True

if already_bought:
    print("Already bought - Hold for more returns:")
    for days, ret in returns.items():
        print("Return in {} days: {:.2f}%".format(days, ret))


Predicted price: 176.90665664672852
Current price: 177.02
SELL - Expected Return: 2.49% 1 week later
Already bought - Hold for more returns:
Return in 1 days: -0.07%
Return in 2 days: 0.02%
Return in 5 days: 0.18%
Return in 7 days: 2.49%


/Users/axjh03/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Users/axjh03/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Users/axjh03/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Users/axjh03/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Users/axjh03/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
